# Horse Racing Bet Builder

This notebook builds comprehensive betting strategies based on predictions from PredictRace.ipynb.

## Prerequisites
- Run PredictRace.ipynb first to generate prediction data
- Ensure the predictions CSV file is available in the current directory

In [588]:
# Import required libraries
import pandas as pd
import numpy as np
import os
from datetime import datetime
import glob
import joblib

print("📊 Horse Racing Bet Builder")
print("" + "="*50)

📊 Horse Racing Bet Builder


In [589]:
# Analyze Historical Spreads to Determine Optimal Horse Count
def analyze_spreads_success_rates():
    """
    Analyze historical data to find spreads with 75% or greater success rates.
    Returns optimal horse counts based on spread ranges for data-driven betting.
    
    This function:
    1. Loads historical race data and model for the current track
    2. Generates predictions on historical races
    3. Analyzes different horse count strategies (1-4 horses) for various spread ranges
    4. Returns strategies that achieve ≥75% success rate
    
    Returns:
        dict: Optimal strategies by spread category, or None if no historical data
    """
    # First check if historical data exists for this track
    try:
        print(f"🔍 Analyzing historical spreads for {track_name}...")
        
        historical_data = pd.read_csv(f'Imputed Data\\{track_name}.csv')
        print(f"📊 Loaded {len(historical_data)} historical entries")
        
        # Load the model and make historical predictions
        model_path = f"Models\\{track_name}\\{track_name}_Model.pkl"
        model = joblib.load(model_path)
        print(f"🤖 Loaded model from {model_path}")
        
        # Get feature names from the model
        feature_names = model.feature_names_in_
        
        # Prepare historical features
        historical_features = historical_data.drop(columns=['normalized_position', 'Position', 'odds'])
        historical_actual = historical_data['Position'].astype(int)
        
        # Generate predictions
        historical_predictions = model.predict(historical_features[feature_names])
        historical_predicted_positions = (historical_predictions * historical_features['number_of_run']) / 100
        print(f"🎯 Generated predictions for {len(historical_predictions)} races")
        
        # Create analysis dataframe
        variance_df = pd.DataFrame({
            'race_id': historical_features['race_id'],
            'actual_position': historical_actual,
            'predicted_position': historical_predicted_positions,
            'field_size': historical_features['number_of_run']
        })
        
        unique_races = variance_df['race_id'].nunique()
        print(f"📈 Analyzing {unique_races} unique historical races")
        
        # Analyze spreads and success rates
        spread_analysis = []
        
        for race_id in variance_df['race_id'].unique():
            race_data = variance_df[variance_df['race_id'] == race_id].sort_values('predicted_position')
            
            if len(race_data) >= 2:
                top_pick = race_data.iloc[0]
                second_pick = race_data.iloc[1]
                spread = second_pick['predicted_position'] - top_pick['predicted_position']
                
                # Analyze different horse count strategies
                strategies = {
                    1: race_data.iloc[0]['actual_position'] == 1,  # Top pick wins
                    2: min(race_data.iloc[:2]['actual_position']) <= 1,  # Top 2 contain winner
                    3: min(race_data.iloc[:3]['actual_position']) <= 1 if len(race_data) >= 3 else False,  # Top 3 contain winner
                    4: min(race_data.iloc[:4]['actual_position']) <= 1 if len(race_data) >= 4 else False,   # Top 4 contain winner
                    5: min(race_data.iloc[:5]['actual_position']) <= 1 if len(race_data) >= 5 else False,   # Top 5 contain winner
                    6: min(race_data.iloc[:6]['actual_position']) <= 1 if len(race_data) >= 6 else False,   # Top 6 contain winner
                    7: min(race_data.iloc[:7]['actual_position']) <= 1 if len(race_data) >= 7 else False,   # Top 7 contain winner
                }
                
                spread_analysis.append({
                    'race_id': race_id,
                    'spread': spread,
                    'field_size': race_data.iloc[0]['field_size'],
                    'strategies': strategies
                })
        
        # Create spread ranges and analyze success rates
        spread_ranges = [
            (0.0, 0.1, "Miniscule Close"),
            (0.1, 0.3, "Very Close"),
            (0.3, 0.6, "Close"), 
            (0.6, 1.0, "Moderate"),
            (1.0, 1.5, "Wide"),
            (1.5, float('inf'), "Very Wide")
        ]
        
        optimal_horses = {}
        
        print(f"🎲 Testing strategies across {len(spread_ranges)} spread categories...")
        
        for min_spread, max_spread, label in spread_ranges:
            matching_races = [r for r in spread_analysis if min_spread <= r['spread'] < max_spread]
            
            if len(matching_races) < 5:  # Need minimum sample size
                print(f"⚠️  {label}: Only {len(matching_races)} races (need ≥5), skipping...")
                continue
            
            print(f"📊 {label} ({min_spread:.1f}-{max_spread:.1f}): {len(matching_races)} races")
            
            best_strategy = None
            best_success_rate = 0
            success_rates = []  # Track all success rates for logging
            
            for horse_count in [1, 2, 3, 4, 5, 6, 7]:
                successes = sum(1 for r in matching_races if r['strategies'][horse_count])
                success_rate = (successes / len(matching_races)) * 100
                success_rates.append(f"{horse_count}H:{success_rate:.1f}%")
                
                # Select the LOWEST number of horses that achieves ≥75% success rate
                if success_rate >= 85 and best_strategy is None:
                    best_success_rate = success_rate
                    best_strategy = horse_count
                    break  # Stop at first (lowest) horse count that meets threshold
            
            # Log all success rates for transparency
            print(f"   Success rates: {' | '.join(success_rates)}")
            
            if best_strategy:
                optimal_horses[label] = {
                    'horses': best_strategy,
                    'success_rate': best_success_rate,
                    'spread_range': (min_spread, max_spread)
                }
                print(f"   ✅ OPTIMAL: Use {best_strategy} horses ({best_success_rate:.1f}% success) - MINIMUM viable strategy")
            else:
                print(f"   ❌ No strategy meets 75% threshold")
        
        print(f"\n🎯 ANALYSIS COMPLETE: Found {len(optimal_horses)} optimal strategies with ≥75% success")
        return optimal_horses
        
    except FileNotFoundError:
        print(f"❌ No historical data found for {track_name}")
        return None
    except Exception as e:
        print(f"❌ Error analyzing spreads: {str(e)}")
        return None

In [590]:
def determine_horses_from_confidence_and_spread(model_confidence, spread, confidence_level, optimal_spreads=None):
    """
    Determine number of horses based on model confidence scores and spread analysis.
    Primary factor: Model confidence score (0-100)
    Secondary factor: Spread between top picks
    """
    # Primary logic based on model confidence scores
    if model_confidence >= 60:  # Very high confidence
        if spread > .5:
            return 1  # Single horse key - very confident with clear separation
        else:
            return 2  # Small coverage - very confident but competitive
    
    elif model_confidence >= 45:  # High confidence
        if spread > 0.2:
            return 2  # Small coverage
        else:
            return 3  # Medium coverage - good confidence but very competitive
    
    elif model_confidence >= 30:  # Moderate confidence
        if spread > .5:
            return 2  # Small coverage - moderate confidence with some separation
        elif spread > 0.3:
            return 3  # Medium coverage
        else:
            return 4  # Large coverage - moderate confidence, very competitive
    
    else:  # Low confidence (< 50)
        if spread > .2:
            return 3  # Medium coverage - low confidence but some separation
        else:
            return 5  # Large coverage - low confidence and competitive


def determine_horses_from_spread_analysis(spread, confidence, optimal_spreads):
    """
    Legacy function - now redirects to confidence-based approach
    """
    # Convert text confidence to estimated model confidence
    confidence_map = {
        'HIGH': 75,
        'MODERATE': 55,
        'LOW': 35
    }
    estimated_confidence = confidence_map.get(confidence, 50)
    
    return determine_horses_from_confidence_and_spread(estimated_confidence, spread, confidence, optimal_spreads)

def get_selection_info(num_horses, spread, confidence, model_confidence=None, data_driven=False):
    """
    Get selection type and reason based on number of horses and confidence.
    """
    if model_confidence is not None:
        conf_info = f"Model Confidence: {model_confidence:.1f}"
        source = f"confidence-based ({conf_info})"
    else:
        source = "spread analysis" if data_driven else "confidence-based"
    
    if num_horses == 1:
        return "KEY", f"Single horse key ({source})"
    elif num_horses == 2:
        return "SMALL", f"Small coverage ({source})"
    elif num_horses == 3:
        return "MEDIUM", f"Medium coverage ({source})"
    elif num_horses <= 5:
        return "LARGE", f"Large coverage ({source})"
    else:
        return "WIDE", f"Wide coverage ({source})"

In [591]:
# Load prediction data from CSV
# Look for the most recent predictions file
track_abbreviation = "cd"
race_date="20250626"

# Use the most recent file
predictions_file = f"Predictions\\predictions_{track_abbreviation}_{race_date}.csv"
print(f"📂 Loading predictions from: {predictions_file}")

# Load the data
results_df = pd.read_csv(predictions_file)

# Check if confidence scores are available
has_confidence = 'confidence_score' in results_df.columns
print(f"📊 Confidence scores available: {'✅ YES' if has_confidence else '❌ NO'}")

if has_confidence:
    print(f"📈 Confidence score range: {results_df['confidence_score'].min():.1f} - {results_df['confidence_score'].max():.1f}")
    print(f"📊 Average confidence: {results_df['confidence_score'].mean():.1f}")
else:
    print("⚠️  Using fallback confidence calculation based on predicted positions")
    # Add fallback confidence scores based on predicted position
    # Lower predicted position = higher confidence
    max_pred = results_df['predicted_finish_position'].max()
    min_pred = results_df['predicted_finish_position'].min()
    if max_pred > min_pred:
        # Invert and normalize: lower prediction = higher confidence
        results_df['confidence_score'] = 100 * (1 - (results_df['predicted_finish_position'] - min_pred) / (max_pred - min_pred))
    else:
        results_df['confidence_score'] = 50  # Default moderate confidence

# Set track name based on abbreviation
track_names = {
    "cd": "Churchill Downs",
    "aq": "Aqueduct",
    "ct": "Charles Town",
    "ke": "Keeneland",
    "pid": "Presque Isle Downs",
    "sa": "Santa Anita",
    "asd": "Assiniboia Downs",
}
track_name = track_names.get(track_abbreviation, track_abbreviation.upper())

print(f"✅ Data loaded successfully!")
print(f"📊 {len(results_df)} entries for {len(results_df['race_id'].unique())} races")
print(f"🏁 Track: {track_name}")
print(f"📅 Date: {race_date}")
print(f"📋 Columns: {', '.join(results_df.columns)}")
print(f"" + "="*50)

📂 Loading predictions from: Predictions\predictions_cd_20250626.csv
📊 Confidence scores available: ✅ YES
📈 Confidence score range: 0.0 - 100.0
📊 Average confidence: 56.3
✅ Data loaded successfully!
📊 80 entries for 8 races
🏁 Track: Churchill Downs
📅 Date: 20250626
📋 Columns: race_id, horse_id, predicted_finish_position, confidence_score


## Betting Configuration

Set your betting parameters here:

In [592]:
# BETTING CONFIGURATION
# ======================
# 🎯 MODIFY THESE PARAMETERS FOR YOUR BETTING STRATEGY

# Basic Settings
base_bet_amount = 1      # Base bet amount in dollars

# Multi-Race Exotic Wager Settings
starting_race_number = 1    # Which race to start multi-race wagers
number_of_races = 8         # How many consecutive races to include

print(f"🎲 BET BUILDER CONFIGURATION")
print(f"{'='*40}")
print(f"Base Bet Amount: ${base_bet_amount:.2f}")
print(f"Multi-Race Sequence: Races {starting_race_number}-{starting_race_number + number_of_races - 1}")
print(f"{'='*40}")

🎲 BET BUILDER CONFIGURATION
Base Bet Amount: $1.00
Multi-Race Sequence: Races 1-8


# Betting Functions

Core functions for analyzing races and building betting strategies.

In [593]:
# Multi-Race Betting Function
def generate_multi_race_bets(start_race, num_races, base_amount=2.00):
    """
    Generate specific betting recommendations for multi-race wagers
    """
    
    # Get all available races from the actual data
    all_race_ids = results_df['race_id'].unique()
    all_races = []
    race_id_map = {}
    
    for race_id in all_race_ids:
        race_num = int(race_id.split('_')[1])
        all_races.append(race_num)
        race_id_map[race_num] = race_id
    
    all_races.sort()
    
    # Validate input parameters
    if start_race not in all_races:
        print(f"❌ ERROR: Race {start_race} not found in today's card")
        print(f"Available races: {all_races}")
        return None, 0
    
    if start_race + num_races - 1 > max(all_races):
        print(f"❌ ERROR: Not enough races available")
        print(f"Requested: Races {start_race}-{start_race + num_races - 1}")
        print(f"Available: Races {min(all_races)}-{max(all_races)}")
        return None, 0
    
    # Get the target races
    target_races = list(range(start_race, start_race + num_races))
    
    print(f"\n🎯 TARGET RACES: {target_races}")
    print(f"{'='*60}")
    
    # Analyze each target race
    race_analysis = []
    total_confidence_score = 0
    
    for race_num in target_races:
        race_id = race_id_map[race_num]  # Use the actual race ID from the data
        race_data = results_df[results_df['race_id'] == race_id].sort_values('predicted_finish_position')
        
        if len(race_data) == 0:
            print(f"❌ No data for Race {race_num}")
            continue
            
        # Get top picks
        top_pick = race_data.iloc[0]
        second_pick = race_data.iloc[1] if len(race_data) > 1 else None
        
        # Extract horse info
        horse_name = top_pick['horse_id'].split('_')[0]
        prog_num = top_pick['horse_id'].split('_')[1]
        predicted_pos = top_pick['predicted_finish_position']
        
        # Determine confidence level
        if predicted_pos <= 1.5:
            confidence = "HIGH"
            confidence_score = 3
        elif predicted_pos <= 2.5:
            confidence = "MODERATE"
            confidence_score = 2
        else:
            confidence = "LOW"
            confidence_score = 1
            
        total_confidence_score += confidence_score
        
        # Second pick info
        second_info = ""
        spread = 0
        if second_pick is not None:
            second_horse = second_pick['horse_id'].split('_')[0]
            second_prog = second_pick['horse_id'].split('_')[1]
            spread = second_pick['predicted_finish_position'] - predicted_pos
            second_info = f"#{second_prog} {second_horse} (Pred: {second_pick['predicted_finish_position']:.2f})"
        
        race_analysis.append({
            'race_num': race_num,
            'race_id': race_id,
            'top_pick': f"#{prog_num} {horse_name}",
            'predicted_pos': predicted_pos,
            'confidence': confidence,
            'confidence_score': confidence_score,
            'second_pick': second_info,
            'spread': spread,
            'field_size': len(race_data)
        })
        
        print(f"\n🏁 RACE {race_num} - {confidence} CONFIDENCE")
        print(f"   Top Pick: #{prog_num} {horse_name} (Predicted: {predicted_pos:.2f})")
        if second_pick is not None:
            print(f"   2nd Pick: {second_info} (Spread: {spread:.2f})")
        print(f"   Field Size: {len(race_data)} horses")
    
    return race_analysis, total_confidence_score

In [594]:
# All-Races Analysis Function
def analyze_all_races(base_amount=2.00):
    """
    Analyze all races for win bets and exacta opportunities using model confidence scores
    """
    
    # Get all available races and analyze each one
    all_race_ids = results_df['race_id'].unique()
    all_races_data = []

    for race_id in sorted(all_race_ids):
        race_num = int(race_id.split('_')[1])
        race_data = results_df[results_df['race_id'] == race_id].sort_values('predicted_finish_position')
        
        if len(race_data) == 0:
            continue
            
        # Get top picks
        top_pick = race_data.iloc[0]
        second_pick = race_data.iloc[1] if len(race_data) > 1 else None
        third_pick = race_data.iloc[2] if len(race_data) > 2 else None
        
        # Extract horse info
        top_horse = top_pick['horse_id'].split('_')[0]
        top_prog = top_pick['horse_id'].split('_')[1]
        top_pred = top_pick['predicted_finish_position']
        
        # Use model confidence scores instead of position-based confidence
        if 'confidence_score' in race_data.columns:
            top_model_confidence = top_pick['confidence_score']
            
            # Determine confidence level based on model confidence scores
            if top_model_confidence >= 75:
                confidence = "HIGH"
                win_confidence = "STRONG"
            elif top_model_confidence >= 50:
                confidence = "MODERATE" 
                win_confidence = "GOOD"
            else:
                confidence = "LOW"
                win_confidence = "WEAK"
        else:
            # Fallback to position-based confidence if no model confidence available
            if top_pred <= 1.5:
                confidence = "HIGH"
                win_confidence = "STRONG"
                top_model_confidence = 80  # Estimated
            elif top_pred <= 2.5:
                confidence = "MODERATE" 
                win_confidence = "GOOD"
                top_model_confidence = 60  # Estimated
            else:
                confidence = "LOW"
                win_confidence = "WEAK"
                top_model_confidence = 30  # Estimated
        
        # Exacta analysis - now use average confidence of top 2 picks
        exacta_suitable = False
        exacta_info = ""
        spread = 0
        top_two_avg_confidence = top_model_confidence
        
        if second_pick is not None:
            second_horse = second_pick['horse_id'].split('_')[0]
            second_prog = second_pick['horse_id'].split('_')[1]
            second_pred = second_pick['predicted_finish_position']
            spread = second_pred - top_pred
            
            # Calculate average confidence of top 2 picks
            if 'confidence_score' in race_data.columns:
                second_model_confidence = second_pick['confidence_score']
                top_two_avg_confidence = (top_model_confidence + second_model_confidence) / 2
            
            # Exacta suitability based on both spread and confidence
            if spread < 1.0 and top_two_avg_confidence >= 60:  # Close race with good confidence
                exacta_suitable = True
                exacta_info = f"#{top_prog}-{second_prog} Box (Spread: {spread:.2f}, Conf: {top_two_avg_confidence:.1f})"
            elif spread < 0.5 and top_two_avg_confidence >= 40:  # Very close race with moderate confidence
                exacta_suitable = True
                exacta_info = f"#{top_prog}-{second_prog} Box (Very Close: {spread:.2f}, Conf: {top_two_avg_confidence:.1f})"
            else:
                exacta_info = f"Wide spread ({spread:.2f}) or low confidence ({top_two_avg_confidence:.1f}) - Consider top pick only"
        
        all_races_data.append({
            'race_num': race_num,
            'race_id': race_id,
            'top_pick': f"#{top_prog} {top_horse}",
            'predicted_pos': top_pred,
            'confidence': confidence,
            'win_confidence': win_confidence,
            'model_confidence': top_model_confidence,
            'top_two_avg_confidence': top_two_avg_confidence,
            'exacta_suitable': exacta_suitable,
            'exacta_info': exacta_info,
            'spread': spread,
            'field_size': len(race_data),
            'second_pick': f"#{second_prog} {second_horse}" if second_pick is not None else None,
            'third_pick': f"#{third_pick['horse_id'].split('_')[1]} {third_pick['horse_id'].split('_')[0]}" if third_pick is not None else None
        })
    
    return all_races_data

In [595]:
# Exotic Wager Builder Function
def build_exotic_wagers(start_race, num_races, base_amount=2.00):
    """
    Build exotic multi-race wagers using model confidence scores as the primary factor.
    Falls back to spread analysis and then hardcoded rules as needed.
    """
    
    # Get the specified races for exotic wagers
    target_races = list(range(start_race, start_race + num_races))
    
    # Get all races data first
    all_races_data = analyze_all_races(base_amount)
    
    # Find data for target races
    target_race_data = []
    for race_num in target_races:
        race_info = next((r for r in all_races_data if r['race_num'] == race_num), None)
        if race_info:
            target_race_data.append(race_info)
    
    if len(target_race_data) < 2:
        print("❌ Need at least 2 races for exotic wagers")
        return None, None
    
    # Check if we have model confidence scores
    has_model_confidence = 'model_confidence' in target_race_data[0] and target_race_data[0]['model_confidence'] is not None
    
    if has_model_confidence:
        print("✅ Using model confidence scores as primary strategy factor")
    else:
        print("⚠️  Model confidence not available - using spread analysis fallback")
        # Analyze historical spreads as fallback
        optimal_spreads = analyze_spreads_success_rates()
        if optimal_spreads:
            print(f"✅ Using data-driven approach with {len(optimal_spreads)} optimal strategies")
        else:
            print(f"⚠️  No historical data available - using hardcoded fallback logic")
    
    print(f"\n📋 RACE ANALYSIS FOR EXOTIC WAGERS:")
    print(f"{'-'*50}")
    
    # Analyze each race and determine how many horses to use
    exotic_selections = []
    
    for race_data in target_race_data:
        race_num = race_data['race_num']
        confidence = race_data['confidence']
        spread = race_data['spread']
        model_confidence = race_data.get('model_confidence', None)
        
        # Determine how many horses to use - prioritize model confidence
        if has_model_confidence and model_confidence is not None:
            # Primary approach: Use model confidence scores
            num_horses = determine_horses_from_confidence_and_spread(
                model_confidence, spread, confidence, None
            )
            selection_type, reason = get_selection_info(
                num_horses, spread, confidence, model_confidence, False
            )
            
        elif 'optimal_spreads' in locals() and optimal_spreads:
            # Secondary approach: Use historical spread analysis
            num_horses = determine_horses_from_spread_analysis(spread, confidence, optimal_spreads)
            selection_type, reason = get_selection_info(num_horses, spread, confidence, None, True)
            
        else:
            # Tertiary approach: Use hardcoded logic based on confidence and spread
            if confidence == 'HIGH' and spread > 0.8:
                num_horses = 1
                selection_type = "KEY"
                reason = "High confidence, clear favorite (fallback)"
                
            elif confidence == 'HIGH' and spread <= 0.8:
                num_horses = 2
                selection_type = "SMALL"
                reason = "High confidence but competitive (fallback)"
                
            elif confidence == 'MODERATE' and spread <= 0.7:
                num_horses = 3
                selection_type = "MEDIUM"
                reason = "Moderate confidence, very competitive (fallback)"
                
            elif confidence == 'MODERATE':
                num_horses = 2
                selection_type = "SMALL"
                reason = "Moderate confidence (fallback)"
                
            else:
                num_horses = 4 if race_data['field_size'] >= 8 else 3
                selection_type = "LARGE"
                reason = "Low confidence, need coverage (fallback)"
        
        # Don't exceed field size
        num_horses = min(num_horses, race_data['field_size'])
        
        exotic_selections.append({
            'race_num': race_num,
            'num_horses': num_horses,
            'selection_type': selection_type,
            'reason': reason,
            'confidence': confidence,
            'model_confidence': model_confidence,
            'spread': spread,
            'top_pick': race_data['top_pick'],
            'second_pick': race_data['second_pick'],
            'third_pick': race_data['third_pick']
        })
        
        # Display race analysis
        print(f"Race {race_num}: Use {num_horses} horses ({selection_type}) - {reason}")
        if model_confidence is not None:
            print(f"   Model Confidence: {model_confidence:.1f}, Text: {confidence}, Spread: {spread:.2f}")
        else:
            print(f"   Confidence: {confidence}, Spread: {spread:.2f}")
        
        # Show which horses
        horses_to_show = []
        horses_to_show.append(race_data['top_pick'])
        if num_horses >= 2 and race_data['second_pick']:
            horses_to_show.append(race_data['second_pick'])
        if num_horses >= 3 and race_data['third_pick']:
            horses_to_show.append(race_data['third_pick'])
        if num_horses >= 4:
            # Get 4th choice and beyond
            race_full_data = results_df[results_df['race_id'] == race_data['race_id']].sort_values('predicted_finish_position')
            for i in range(3, min(num_horses, len(race_full_data))):
                pick = race_full_data.iloc[i]
                horse = f"#{pick['horse_id'].split('_')[1]} {pick['horse_id'].split('_')[0]}"
                horses_to_show.append(horse)
        
        print(f"   Horses: {', '.join(horses_to_show)}")
        print()
    
    return exotic_selections, target_race_data

# Betting Strategy Execution

Execute your betting strategies based on the configuration above.

In [596]:
# ALL-RACES WIN & EXACTA ANALYSIS
print(f"\n{'🏆 ALL RACES WIN & EXACTA OPPORTUNITIES'}")
print(f"{'='*70}")

# Get all races data
all_races_data = analyze_all_races(base_bet_amount)

# Display all races analysis
for race_data in all_races_data:
    print(f"\n🏁 RACE {race_data['race_num']} - {race_data['win_confidence']} WIN CONFIDENCE")
    print(f"   Top Pick: {race_data['top_pick']} (Predicted: {race_data['predicted_pos']:.2f})")
    print(f"   Field Size: {race_data['field_size']} horses")
    
    # Win bet recommendation
    win_bet_size = 5
    if race_data['confidence'] == 'HIGH':
        win_bet_size *= 1.5
    elif race_data['confidence'] == 'MODERATE':
        win_bet_size *= 1.2
    
    print(f"   💰 Win Bet: ${win_bet_size:.2f} ({race_data['confidence']} confidence)")
    
    # Exacta recommendation
    if race_data['exacta_suitable']:
        exacta_cost = 1
        print(f"   🎯 Exacta Box: {race_data['exacta_info']} - ${exacta_cost:.2f}")
        print(f"       Top 2: {race_data['top_pick']} / {race_data['second_pick']}")
    else:
        print(f"   ⚠️  Exacta: {race_data['exacta_info']}")

# Summary of recommendations
total_win_cost = sum(base_bet_amount * (1.5 if r['confidence'] == 'HIGH' else 1.2 if r['confidence'] == 'MODERATE' else 1.0) for r in all_races_data)
total_exacta_cost = sum(base_bet_amount * 2 for r in all_races_data if r['exacta_suitable'])
high_confidence_races = len([r for r in all_races_data if r['confidence'] == 'HIGH'])
exacta_opportunities = len([r for r in all_races_data if r['exacta_suitable']])

print(f"\n{'📊 ALL-RACES BETTING SUMMARY'}")
print(f"{'='*50}")
print(f"Total Races: {len(all_races_data)}")
print(f"High Confidence Races: {high_confidence_races}")
print(f"Exacta Opportunities: {exacta_opportunities}")
print(f"Total Win Bets Cost: ${total_win_cost:.2f}")
print(f"Total Exacta Boxes Cost: ${total_exacta_cost:.2f}")
print(f"Combined Investment: ${total_win_cost + total_exacta_cost:.2f}")


🏆 ALL RACES WIN & EXACTA OPPORTUNITIES

🏁 RACE 1 - GOOD WIN CONFIDENCE
   Top Pick: #4 PIERCE ELEVATED (Predicted: 3.15)
   Field Size: 7 horses
   💰 Win Bet: $6.00 (MODERATE confidence)
   🎯 Exacta Box: #4-6 Box (Spread: 0.13, Conf: 65.3) - $1.00
       Top 2: #4 PIERCE ELEVATED / #6 GOLDEN PLATE

🏁 RACE 2 - WEAK WIN CONFIDENCE
   Top Pick: #3 BORN FLASHY (Predicted: 2.51)
   Field Size: 8 horses
   💰 Win Bet: $5.00 (LOW confidence)
   ⚠️  Exacta: Wide spread (0.41) or low confidence (20.4) - Consider top pick only

🏁 RACE 3 - STRONG WIN CONFIDENCE
   Top Pick: #5 ASHES AND DIAMONDS (Predicted: 3.12)
   Field Size: 8 horses
   💰 Win Bet: $7.50 (HIGH confidence)
   🎯 Exacta Box: #5-6 Box (Spread: 0.40, Conf: 76.6) - $1.00
       Top 2: #5 ASHES AND DIAMONDS / #6 ORDER RESTORED

🏁 RACE 4 - GOOD WIN CONFIDENCE
   Top Pick: #5 LORD BULLINGDON (Predicted: 3.61)
   Field Size: 9 horses
   💰 Win Bet: $6.00 (MODERATE confidence)
   🎯 Exacta Box: #5-1 Box (Spread: 0.09, Conf: 82.4) - $1.00
  

In [597]:
# Generate exotic wager recommendations 
print("\n🎲 MULTI-RACE BETTING RECOMMENDATIONS")
print("="*50)
print(f"Starting Race: {starting_race_number}")
print(f"Number of Races: {number_of_races}")  # Use available races
print(f"Base Bet Amount: ${base_bet_amount:.2f}")
print("="*50)

race_analysis, total_confidence = generate_multi_race_bets(starting_race_number, number_of_races, base_bet_amount)

if race_analysis:
    avg_confidence = total_confidence / len(race_analysis)
    if avg_confidence >= 2.5:
        overall_confidence = "HIGH"
    elif avg_confidence >= 1.5:
        overall_confidence = "MODERATE"
    else:
        overall_confidence = "LOW"
    
    print(f"\nOverall Confidence Level: {overall_confidence} (Score: {total_confidence}/{len(race_analysis)*3})")
    print(f"Average Confidence: {avg_confidence:.2f}/3.0")
    print(f"Number of Races: {len(race_analysis)}")
else:
    print("❌ Could not generate recommendations - check race parameters")


🎲 MULTI-RACE BETTING RECOMMENDATIONS
Starting Race: 1
Number of Races: 8
Base Bet Amount: $1.00

🎯 TARGET RACES: [1, 2, 3, 4, 5, 6, 7, 8]

🏁 RACE 1 - LOW CONFIDENCE
   Top Pick: #4 PIERCE ELEVATED (Predicted: 3.15)
   2nd Pick: #6 GOLDEN PLATE (Pred: 3.28) (Spread: 0.13)
   Field Size: 7 horses

🏁 RACE 2 - LOW CONFIDENCE
   Top Pick: #3 BORN FLASHY (Predicted: 2.51)
   2nd Pick: #4 BOURBON OUTLAW (Pred: 2.92) (Spread: 0.41)
   Field Size: 8 horses

🏁 RACE 3 - LOW CONFIDENCE
   Top Pick: #5 ASHES AND DIAMONDS (Predicted: 3.12)
   2nd Pick: #6 ORDER RESTORED (Pred: 3.52) (Spread: 0.40)
   Field Size: 8 horses

🏁 RACE 4 - LOW CONFIDENCE
   Top Pick: #5 LORD BULLINGDON (Predicted: 3.61)
   2nd Pick: #1 BOURBON RESOLVE (Pred: 3.70) (Spread: 0.09)
   Field Size: 9 horses

🏁 RACE 5 - LOW CONFIDENCE
   Top Pick: #4 EXECUTIVE ACTION (Predicted: 3.64)
   2nd Pick: #1 DEAN MARTINI (Pred: 4.25) (Spread: 0.61)
   Field Size: 9 horses

🏁 RACE 6 - LOW CONFIDENCE
   Top Pick: #2 LADY OF SILENCE (Pred

In [598]:
# EXOTIC MULTI-RACE WAGERS CONSTRUCTION
print(f"\n{'🎰 EXOTIC MULTI-RACE WAGERS'}")
print(f"{'='*70}")
print(f"Building exotic wagers for Races {starting_race_number}-{starting_race_number + number_of_races - 1}")

# Build the exotic wager selections
exotic_selections, target_race_data = build_exotic_wagers(starting_race_number, number_of_races, base_bet_amount)

if exotic_selections and len(exotic_selections) >= 2:
    
    # Calculate costs for different exotic wagers
    print(f"\n{'💰 EXOTIC WAGER COSTS & RECOMMENDATIONS'}")
    print(f"{'='*60}")
    
    # Store exotic costs for summary
    exotic_costs = {}
    
    # Daily Double (first 2 races)
    if len(exotic_selections) >= 2:
        dd_combinations = exotic_selections[0]['num_horses'] * exotic_selections[1]['num_horses']
        dd_cost = dd_combinations * base_bet_amount
        exotic_costs['Daily Double'] = dd_cost
        print(f"\n🎯 DAILY DOUBLE (Races {exotic_selections[0]['race_num']}-{exotic_selections[1]['race_num']}):")
        print(f"   Combinations: {exotic_selections[0]['num_horses']} x {exotic_selections[1]['num_horses']} = {dd_combinations}")
        print(f"   Cost: ${dd_cost:.2f} (${base_bet_amount:.2f} per combination)")
        print(f"   Strategy: {exotic_selections[0]['selection_type']} in Race {exotic_selections[0]['race_num']}, {exotic_selections[1]['selection_type']} in Race {exotic_selections[1]['race_num']}")
    
    # Pick 3 (if 3+ races)
    if len(exotic_selections) >= 3:
        p3_combinations = exotic_selections[0]['num_horses'] * exotic_selections[1]['num_horses'] * exotic_selections[2]['num_horses']
        p3_cost = p3_combinations * base_bet_amount
        exotic_costs['Pick 3'] = p3_cost
        print(f"\n🎯 PICK 3 (Races {exotic_selections[0]['race_num']}-{exotic_selections[2]['race_num']}):")
        print(f"   Combinations: {exotic_selections[0]['num_horses']} x {exotic_selections[1]['num_horses']} x {exotic_selections[2]['num_horses']} = {p3_combinations}")
        print(f"   Cost: ${p3_cost:.2f} (${base_bet_amount:.2f} per combination)")
        
        # Show strategy
        strategies = [f"{sel['selection_type']} in Race {sel['race_num']}" for sel in exotic_selections[:3]]
        print(f"   Strategy: {', '.join(strategies)}")
    
    # Pick 4 (if 4+ races)
    if len(exotic_selections) >= 4:
        p4_combinations = 1
        for sel in exotic_selections[:4]:
            p4_combinations *= sel['num_horses']
        p4_cost = p4_combinations * base_bet_amount
        exotic_costs['Pick 4'] = p4_cost
        print(f"\n🎯 PICK 4 (Races {exotic_selections[0]['race_num']}-{exotic_selections[3]['race_num']}):")
        print(f"   Combinations: {' x '.join([str(sel['num_horses']) for sel in exotic_selections[:4]])} = {p4_combinations}")
        print(f"   Cost: ${p4_cost:.2f} (${base_bet_amount:.2f} per combination)")
    
    # Recommendations based on cost and confidence
    print(f"\n{'🎯 EXOTIC WAGER RECOMMENDATIONS'}")
    print(f"{'='*50}")
    
    high_conf_count = sum(1 for sel in exotic_selections if sel['confidence'] == 'HIGH')
    total_confidence = len([sel for sel in exotic_selections if sel['confidence'] in ['HIGH', 'MODERATE']])
    
    if len(exotic_selections) >= 2 and exotic_costs.get('Daily Double', 0) <= base_bet_amount * 10:
        print(f"✅ DAILY DOUBLE: Cost-effective at ${exotic_costs['Daily Double']:.2f}")
    
    if len(exotic_selections) >= 3 and exotic_costs.get('Pick 3', 0) <= base_bet_amount * 20 and high_conf_count >= 1:
        print(f"✅ PICK 3: Good value at ${exotic_costs['Pick 3']:.2f} with {high_conf_count} high-confidence races")
    
    if len(exotic_selections) >= 4 and exotic_costs.get('Pick 4', 0) <= base_bet_amount * 50 and total_confidence >= 3:
        print(f"✅ PICK 4: Reasonable at ${exotic_costs['Pick 4']:.2f} with good overall confidence")
    elif len(exotic_selections) >= 4:
        print(f"⚠️  PICK 4: High cost (${exotic_costs.get('Pick 4', 0):.2f}) - consider smaller ticket")

else:
    print("❌ Need at least 2 races for exotic wagers")
    exotic_costs = {}


🎰 EXOTIC MULTI-RACE WAGERS
Building exotic wagers for Races 1-8
✅ Using model confidence scores as primary strategy factor

📋 RACE ANALYSIS FOR EXOTIC WAGERS:
--------------------------------------------------
Race 1: Use 3 horses (MEDIUM) - Medium coverage (confidence-based (Model Confidence: 52.3))
   Model Confidence: 52.3, Text: MODERATE, Spread: 0.13
   Horses: #4 PIERCE ELEVATED, #6 GOLDEN PLATE, #3 GOLD SEARCH

Race 2: Use 3 horses (MEDIUM) - Medium coverage (confidence-based (Model Confidence: 0.0))
   Model Confidence: 0.0, Text: LOW, Spread: 0.41
   Horses: #3 BORN FLASHY, #4 BOURBON OUTLAW, #6 MUGATU

Race 3: Use 2 horses (SMALL) - Small coverage (confidence-based (Model Confidence: 100.0))
   Model Confidence: 100.0, Text: HIGH, Spread: 0.40
   Horses: #5 ASHES AND DIAMONDS, #6 ORDER RESTORED

Race 4: Use 2 horses (SMALL) - Small coverage (confidence-based (Model Confidence: 69.6))
   Model Confidence: 69.6, Text: MODERATE, Spread: 0.09
   Horses: #5 LORD BULLINGDON, #1 BO

In [599]:
# COMPREHENSIVE BETTING STRATEGY SUMMARY
print(f"\n{'🎯 COMPLETE BETTING STRATEGY SUMMARY'}")
print(f"{'='*70}")

# Calculate total investments for different approaches
all_races_investment = total_win_cost + total_exacta_cost

print(f"\n💰 INVESTMENT OPTIONS COMPARISON:")
print(f"{'-'*50}")

# Option 1: All races win + exacta
print(f"📋 OPTION 1: ALL RACES WIN & EXACTA")
print(f"   Win bets on all {len(all_races_data)} races: ${total_win_cost:.2f}")
print(f"   Exacta boxes on {exacta_opportunities} races: ${total_exacta_cost:.2f}")
print(f"   Total Investment: ${all_races_investment:.2f}")
print(f"   Strategy: Comprehensive coverage, moderate risk")

# Option 2: Specified races exotic wagers
if 'exotic_costs' in locals() and exotic_costs:
    print(f"\n📋 OPTION 2: MULTI-RACE EXOTICS (Races {starting_race_number}-{starting_race_number + number_of_races - 1})")
    for bet_type, cost in exotic_costs.items():
        print(f"   {bet_type}: ${cost:.2f}")
    
    min_exotic_cost = min(exotic_costs.values())
    max_exotic_cost = max(exotic_costs.values())
    print(f"   Investment Range: ${min_exotic_cost:.2f} - ${max_exotic_cost:.2f}")
    print(f"   Strategy: High-risk, high-reward exotic wagers")

# Option 3: Combination approach
if 'exotic_costs' in locals() and exotic_costs:
    # Suggest win bets on high-confidence races + one exotic wager
    high_conf_win_cost = sum(base_bet_amount * 1.5 for r in all_races_data if r['confidence'] == 'HIGH')
    recommended_exotic = min(exotic_costs.items(), key=lambda x: x[1])  # Cheapest exotic
    combo_cost = high_conf_win_cost + recommended_exotic[1]
    
    print(f"\n📋 OPTION 3: COMBINATION APPROACH")
    print(f"   Win bets on {high_confidence_races} high-confidence races: ${high_conf_win_cost:.2f}")
    print(f"   {recommended_exotic[0]}: ${recommended_exotic[1]:.2f}")
    print(f"   Total Investment: ${combo_cost:.2f}")
    print(f"   Strategy: Balanced risk, diversified approach")

# Final recommendation based on bankroll and risk tolerance
print(f"\n🎯 RECOMMENDED STRATEGY:")
print(f"{'-'*30}")

if 'exotic_costs' in locals() and exotic_costs:
    # If we have affordable exotic options
    daily_double_cost = exotic_costs.get('Daily Double', float('inf'))
    pick3_cost = exotic_costs.get('Pick 3', float('inf'))
    
    if daily_double_cost <= base_bet_amount * 6:  # Reasonable DD cost
        print(f"✅ CONSERVATIVE: Daily Double (${daily_double_cost:.2f}) + Win bets on high-confidence races")
        print(f"   Total: ${daily_double_cost + high_conf_win_cost:.2f}")
    elif pick3_cost <= base_bet_amount * 20:  # Reasonable P3 cost
        print(f"✅ MODERATE: Pick 3 (${pick3_cost:.2f}) for big score potential")
    else:
        print(f"✅ CONSERVATIVE: Focus on win bets and exacta boxes")
        print(f"   High-confidence wins + exactas: ${high_conf_win_cost + (exacta_opportunities * base_bet_amount * 2):.2f}")
else:
    print(f"✅ FOCUS ON: Win bets and exacta boxes")
    print(f"   Total investment: ${all_races_investment:.2f}")

print(f"\n💡 TIPS:")
print(f"   • Start with smaller amounts to test the strategy")
print(f"   • Focus on races with HIGH confidence ratings")
print(f"   • Exacta boxes work best when spread < 1.0")
print(f"   • Multi-race exotics are high-risk, high-reward")
print(f"   • Adjust base_bet_amount to fit your bankroll")

print(f"\n🔄 TO MODIFY: Change the configuration parameters above and re-run!")
print(f"   - starting_race_number: {starting_race_number}")
print(f"   - number_of_races: {number_of_races}")
print(f"   - base_bet_amount: ${base_bet_amount:.2f}")

print(f"\n💡 Change the parameters in the configuration section and re-run to test different combinations!")


🎯 COMPLETE BETTING STRATEGY SUMMARY

💰 INVESTMENT OPTIONS COMPARISON:
--------------------------------------------------
📋 OPTION 1: ALL RACES WIN & EXACTA
   Win bets on all 8 races: $9.30
   Exacta boxes on 4 races: $8.00
   Total Investment: $17.30
   Strategy: Comprehensive coverage, moderate risk

📋 OPTION 2: MULTI-RACE EXOTICS (Races 1-8)
   Daily Double: $9.00
   Pick 3: $18.00
   Pick 4: $36.00
   Investment Range: $9.00 - $36.00
   Strategy: High-risk, high-reward exotic wagers

📋 OPTION 3: COMBINATION APPROACH
   Win bets on 1 high-confidence races: $1.50
   Daily Double: $9.00
   Total Investment: $10.50
   Strategy: Balanced risk, diversified approach

🎯 RECOMMENDED STRATEGY:
------------------------------
✅ MODERATE: Pick 3 ($18.00) for big score potential

💡 TIPS:
   • Start with smaller amounts to test the strategy
   • Focus on races with HIGH confidence ratings
   • Exacta boxes work best when spread < 1.0
   • Multi-race exotics are high-risk, high-reward
   • Adjust